In [1]:
import pandas as pd
import json as js

with open("/u/pmpande/author_profiling/fb.json") as f:
    data_essays = pd.DataFrame(js.loads(line) for line in f)
f.close()

print(len(data_essays))

9917


In [2]:
data_essays.drop(data_essays.head(1).index, inplace=True)

In [3]:
NRCDict = {}
with open("NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt",'r') as f:
    for line in f:
        text = line.split()
        if text[2] == '1':
            if text[0] in NRCDict:
                NRCDict[text[0]] += "#" + text[1]
            else:
                NRCDict[text[0]] = text[1]

In [4]:
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree
from empath import Empath
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
import nltk
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

accuracy = 0.0
rmse = 0.0
h_loss = 0.0
for i in range(10):
    rand_essays = []
    temp_data = data_essays.copy()
    rand_essays = random.sample(range(1,len(temp_data)-1),1000)
    test = temp_data.iloc[rand_essays].copy()
    train = temp_data.drop(rand_essays)
    train.drop(train.columns[[1,7,8,9,10,11]], axis=1, inplace=True)
    test.drop(test.columns[[1,7,8,9,10,11]], axis=1, inplace=True)
    
    corpus = []
    truth = []
    for row in train.iterrows():
        Y = [0,0,0,0,0]
        corpus.append(row[1]['STATUS'])
        if row[1]['AGR'] == 'y': Y[0] = 1
        if row[1]['CON'] == 'y': Y[1] = 1
        if row[1]['EXT'] == 'y': Y[2] = 1
        if row[1]['NEU'] == 'y': Y[3] = 1
        if row[1]['OPN'] == 'y': Y[4] = 1
        truth.append(Y)
    truth = np.array(truth)
    feat = []
    for row in corpus:
        tokens = nltk.word_tokenize(row)
        feat_row = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
        for word in tokens:
            if word in NRCDict:
                category = NRCDict[word].split("#")
                if "anger" in category: feat_row[0] += 1
                if "anticipation" in category: feat_row[1] += 1
                if "disgust" in category: feat_row[2] += 1
                if "fear" in category: feat_row[3] += 1
                if "joy" in category: feat_row[4]+= 1
                if "negative" in category: feat_row[5] += 1
                if "positive" in category: feat_row[6] += 1
                if "sadness" in category: feat_row[7] += 1
                if "surprise" in category: feat_row[8] += 1
                if "trust" in category: feat_row[9] += 1
            else: continue
        feat.append(feat_row)
    feat = np.array(feat)
    feat_norm = normalize(feat, axis=1, norm='l1')
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(feat_norm, truth)
    
    test_data = []
    truth_test = []
    for row in test.iterrows():
        Y = [0,0,0,0,0]
        test_data.append(row[1]['STATUS'])
        if row[1]['AGR'] == 'y': Y[0] = 1
        if row[1]['CON'] == 'y': Y[1] = 1
        if row[1]['EXT'] == 'y': Y[2] = 1
        if row[1]['NEU'] == 'y': Y[3] = 1
        if row[1]['OPN'] == 'y': Y[4] = 1
        truth_test.append(Y)
    truth_test = np.array(truth_test)
    test_feat = []
    for row in test_data:
        tokens = nltk.word_tokenize(row)
        feat_row = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
        for word in tokens:
            if word in NRCDict:
                category = NRCDict[word].split("#")
                if "anger" in category: feat_row[0] += 1
                if "anticipation" in category: feat_row[1] += 1
                if "disgust" in category: feat_row[2] += 1
                if "fear" in category: feat_row[3] += 1
                if "joy" in category: feat_row[4] += 1
                if "negative" in category: feat_row[5] += 1
                if "positive" in category: feat_row[6] += 1
                if "sadness" in category: feat_row[7] += 1
                if "surprise" in category: feat_row[8] += 1
                if "trust" in category: feat_row[9] += 1
            else: continue
        test_feat.append(feat_row)
    test_feat = np.array(test_feat)
    test_feat_norm = normalize(test_feat, axis=1, norm='l1')
    
    output = clf.predict(test_feat_norm)
    err = sqrt(mean_squared_error(truth_test, output))
    acc = accuracy_score(truth_test, output, normalize=True)
    loss = hamming_loss(truth_test, output)
    accuracy += acc
    rmse += err
    h_loss += loss
    print("Accuracy for interation " + str(i) + ": " + str(acc))
    print("Hamming Loss for interation " + str(i) + ": " + str(loss))
    print("RMSE for interation " + str(i) + ": " + str(err))
    del test
    del train
    del temp_data
print("-------------------------------")
print("Average accuracy is :" + str(accuracy/10))
print("Average haming loss is :" + str(h_loss/10))
print("Average RMSE loss is :" + str(rmse/10))

Accuracy for interation 0: 0.225
Hamming Loss for interation 0: 0.328
RMSE for interation 0: 0.5727128425310541
Accuracy for interation 1: 0.194
Hamming Loss for interation 1: 0.3452
RMSE for interation 1: 0.5875372328627353
Accuracy for interation 2: 0.214
Hamming Loss for interation 2: 0.32
RMSE for interation 2: 0.565685424949238
Accuracy for interation 3: 0.218
Hamming Loss for interation 3: 0.3172
RMSE for interation 3: 0.563205113613149
Accuracy for interation 4: 0.205
Hamming Loss for interation 4: 0.3332
RMSE for interation 4: 0.5772347875864725
Accuracy for interation 5: 0.218
Hamming Loss for interation 5: 0.3324
RMSE for interation 5: 0.576541412215983
Accuracy for interation 6: 0.194
Hamming Loss for interation 6: 0.3464
RMSE for interation 6: 0.5885575587824864
Accuracy for interation 7: 0.23
Hamming Loss for interation 7: 0.3228
RMSE for interation 7: 0.5681549084536717
Accuracy for interation 8: 0.231
Hamming Loss for interation 8: 0.3296
RMSE for interation 8: 0.5741080